In [1]:
!pip list | grep lang

langchain                0.0.263
langchainplus-sdk        0.0.17
langsmith                0.0.22


In [2]:
!pip list | grep openai

openai                   0.27.8


### LangChain Expression Language
In this notebook we will have a try on the latest feature released by langchain, using pipeline to link model, prompt and any function or parser you want.


#### 1. Toy example
We will first take a look at a simplest example to link a LLM and a prompt. Before we take a look at Langchain expression language, let's take a look at pipe package which support pipe operation in python

In [4]:
from pipe import select, where

numbers = [1, 2, 3, 4, 5]
# find even numbers and multiple by 3
results = list(numbers |  where(lambda x: x % 2 == 0) | select(lambda x: x * 3))

results

[6, 12]

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from dotenv import load_dotenv

# load env for llm service
load_dotenv()

True

In [6]:
# create llm
llm = AzureChatOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')

# create prompt template
prompt = ChatPromptTemplate.from_template("Show me description for company {company_name} in 100 words.")

# chain model and prompt
chain = prompt | llm

chain

RunnableSequence(first=ChatPromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, template='Show me description for company {company_name} in 100 words.', template_format='f-string', validate_template=True), additional_kwargs={})]), middle=[], last=AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='d2d65893138c46d39ffe1d69f52eda88', openai_api_base='https://qucy-openai-test.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='chatGPTAzure', model_version='', openai_api_type='azure', openai_api

In [7]:
chain.invoke({"company_name": "HSBC"})

AIMessage(content="HSBC is a global banking and financial services company headquartered in London, UK. With a presence in 64 countries and territories, HSBC serves millions of customers worldwide through its retail banking, wealth management, commercial banking, and global banking and markets services. The company's aim is to provide customers with simple, efficient, and secure banking solutions that help them to achieve their financial goals. HSBC is committed to sustainability and social responsibility, and works to reduce its environmental impact and support local communities through various initiatives.", additional_kwargs={}, example=False)

In [8]:
# a chain with a string output parser
chain_with_output_parser = prompt | llm | StrOutputParser()
chain_with_output_parser

RunnableSequence(first=ChatPromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, template='Show me description for company {company_name} in 100 words.', template_format='f-string', validate_template=True), additional_kwargs={})]), middle=[AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='d2d65893138c46d39ffe1d69f52eda88', openai_api_base='https://qucy-openai-test.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='chatGPTAzure', model_version='', openai_api_type='azure', openai_api_version

In [9]:
chain_with_output_parser.invoke({"company_name": "HSBC"})

'HSBC is a multinational banking and financial services company headquartered in London, United Kingdom. It is one of the largest banks in the world with operations in more than 65 countries. HSBC offers a wide range of financial products and services such as personal and commercial banking, wealth management, investment banking, and insurance. The company is committed to providing excellent customer service and operates with a focus on sustainability and corporate responsibility. HSBC is known for its strong presence in emerging markets and its ability to help clients navigate complex global business environments.'

#### 2. Attaching Function Call information

We are not call the function acctually here, we are just using the function specification and let LLM to figure out the input parameter for the function

In [10]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# functions = [
#     {
#       "name": "save_description",
#       "description": "save the description for the a company",
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "company_name": {
#             "type": "string",
#             "description": "The name of the company"
#           },
#           "description": {
#             "type": "string",
#             "description": "The description of the company"
#           }
#         },
#         "required": ["company_name", "description"]
#       }
#     }
#   ]


# chain = (
#     prompt 
#     | llm.bind(function_call= {"name": "save_description"}, functions= functions) 
#     | JsonOutputFunctionsParser()
# )

# chain.invoke({"company_name": "HSBC"})

The output should be look like, bue since our OPENAI model at Azure is still the old GPT 3.5 which does not support functional call and it will encounter an error.

{ 
  'company_name' : 'HSBC',
  'description' : 'HSBC Holdings plc is a multinational banking and financial services company headquartered in London, United Kingdom. It is one of the largest banking organizations in the world, with a presence in 64 countries and territories, serving more than 40 million customers. HSBC offers a wide range of......'
}

### 3. Runnable in Langchain

LangChain Expression Language (LEL) provides a convenient and straightforward approach for writing components in a declarative manner and effortlessly combining them. To gain a better understanding of its functionality, let's examine its runnable interface.

LangChain introduces a runnable interface that serves as a blueprint for implementing various components. Subclasses of this interface are required to implement the following functions:
- stream: handles streaming output
- invoke: processes single input
- batch: handles batch input
- astream: handles asynchronous streaming
- ainvoke: performs asynchronous invocation
- abatch: handles asynchronous batch processing

In [11]:
from langchain.schema.runnable import Runnable

# the ChatPromptTemplate implemented Runnable as well
print(isinstance(prompt, Runnable))

True


In [12]:
# let's write our own component

from langchain.load.serializable import Serializable
from langchain.schema.runnable import RunnableConfig, RunnablePassthrough
from langchain.schema.runnable.base import Input, Runnable, Optional


class StdOutputRunnable(Serializable, Runnable[Input, Input]):

    @property
    def lc_serializable(self) -> bool:
        return True

    def invoke(self, input: Input, config: Optional[RunnableConfig] = None) -> Input:
        print(f"I recevied input ->  {input} ")
        return self._call_with_config(lambda x: x, input, config)


runnable_chain =  RunnablePassthrough() | StdOutputRunnable()

runnable_chain.invoke({'company_name':'HSBC'})

I recevied input ->  {'company_name': 'HSBC'} 


{'company_name': 'HSBC'}

In [13]:
chain = prompt | StdOutputRunnable() | llm

chain.invoke({'company_name': 'Alibaba'})

I recevied input ->  messages=[HumanMessage(content='Show me description for company Alibaba in 100 words.', additional_kwargs={}, example=False)] 


AIMessage(content="Alibaba Group is a leading e-commerce company based in Hangzhou, China, founded by Jack Ma in 1999. The company initially started as a B2B marketplace, connecting Chinese manufacturers to global buyers. Since then, it has expanded its services to include B2C retail, cloud computing, digital media, and entertainment. Alibaba's e-commerce platforms, including Taobao and Tmall, are some of the largest in the world, with millions of active users. The company is also a major player in the cloud computing market, providing services to businesses across various industries. Alibaba's mission is to make it easy to do business anywhere, and it continues to innovate and expand its offerings to achieve this goal.", additional_kwargs={}, example=False)

#### 4. Retriever

In [14]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

# create embedding model
embeddings = OpenAIEmbeddings(
    deployment="text-embedding-ada-002",
    model="text-embedding-ada-002"
)

# Create the retriever
vectorstore = Chroma.from_texts(["harrison worked at kensho"], embedding=embeddings)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

In [15]:
chain.invoke("where did harrison work?")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'Harrison worked at Kensho.'

### 5. Router

In [16]:
from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field
from langchain.schema.runnable import RouterRunnable


class ChainToUse(BaseModel):
    """Used to determine which chain to use to answer the user's question."""
    
    name: str = Field(description="Should be one of `market_view`, `asset_view`")

# create tagger to do classification
tagger = create_tagging_chain_pydantic(ChainToUse, llm)

# create 2 seperate chains
asset_view_chain = ChatPromptTemplate.from_template(""" 
You are a financial expert to answer question related to asset view, 
- you should involves analyzing and evaluating specific assets or financial instruments held by the bank. 
- you should focuses on individual investments, such as stocks, bonds, loans, real estate, or other financial products. 
- you should cover factors like asset quality, performance, risk, liquidity, and potential returns. 
Answer the question: {question}
""") | llm

market_view_chain = ChatPromptTemplate.from_template(""" 
You are a financial expert to answer question related to market view, 
- you should analysis or perspective focused on the broader financial markets, including various market segments such as stocks, bonds, commodities, currencies, and derivatives. 
- you should involves assessing market conditions, trends, and factors that may impact the overall performance of these markets. 
Answer the question: {question}
""") | llm

# create a router
router = RouterRunnable({"asset_view": asset_view_chain, "market_view": market_view_chain})

# using LEL to chain everything together
chain = {
    "key": {"input": lambda x: x["question"]} | tagger | (lambda x: x['text'].name),
    "input": {"question": lambda x: x["question"]}
} | router

In [18]:
news = """
Headline: "Banking Sector Anticipates Robust Growth in Real Estate Investments Amidst Economic Recovery"
In the wake of a solid economic recovery, the banking sector is gearing up for a surge in real estate investments. 
Market analysts predict that the rebounding economy, low interest rates, 
and increased consumer confidence will drive demand for residential and commercial properties. 
This optimistic market view has prompted several banks to strategically expand their real estate portfolios. 
Simultaneously, asset managers within these banks are eyeing lucrative opportunities in key metropolitan areas, 
identifying undervalued properties with growth potential. With a focus on diversifying their asset holdings and capitalizing on the promising market conditions, 
banks are poised to leverage the recovering real estate sector for long-term profitability and asset appreciation."""

#chain.invoke({"question": f"What is the asset view for news {news} "})

In [19]:
# chain.invoke({"question": f"What is the market view for news {news} "})